In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import torch
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler, RobustScaler
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from torchmetrics.classification import AUROC, Accuracy, ConfusionMatrix, F1Score
import os, subprocess, gc, time, datetime
from itertools import product

import models.original_models as original_models
import models.models_3d_atomics_on_variate_to_concepts as new_models
from vasopressor.preprocess_helpers import load_and_create_MIMIC_dataloader
from models.helper import *
from models.param_initializations import *
from models.optimization_strategy import greedy_selection

gpu_id = int(subprocess.check_output('nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader | nl -v 0 | sort -nrk 2 | cut -f 1 | head -n 1 | xargs', shell=True, text=True))
device = torch.device(f'cuda:{gpu_id}') if torch.cuda.is_available else torch.device('cpu')
print("current device", device)

current device cuda:15


In [2]:
train_loader, val_loader, test_loader, class_weights, num_classes, changing_vars, static_names, seq_len = load_and_create_MIMIC_dataloader(output_dim = 2, batch_size = 512, random_state = 1)

print(class_weights, num_classes, seq_len)

for batch in train_loader:
    [print(t.shape, t.device) for t in batch]
    break

len(train_loader)

tensor([0.5797, 3.6376], dtype=torch.float64) 2 6
torch.Size([512, 6, 27]) cpu
torch.Size([512, 6, 27]) cpu
torch.Size([512, 8]) cpu
torch.Size([512, 2]) cpu


35

In [3]:
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, color="black", label="Train")
    plt.plot(val_losses, color="green", label="Val")
    plt.yscale("log")
    plt.legend()
    plt.show()

def plot_metrics(history, n_concepts_list):
    plt.plot(history[:, 0], history[:, 2], label=f'AUC')
    plt.plot(history[:, 0], history[:, 3], label=f'ACC')
    plt.plot(history[:, 0], history[:, 4], label=f'F1')

    plt.xlabel('Num Concepts')
    plt.ylabel('Criteria')
    plt.title('Plot of Concepts vs Criteria')
    plt.xticks(n_concepts_list)

    plt.legend()
    plt.show()

def plot_atomics_concepts_metric(history, title, dec="{:.3g}"):
        
    df = pd.DataFrame(history, columns=["n_atomics", "n_concepts", "val_loss", "auc", "acc", "f1"])
    mean_atomics = df.groupby("n_atomics").mean()
    mean_concepts = df.groupby("n_concepts").mean()

    # display(mean_atomics)
    plt.plot(mean_atomics.index, mean_atomics["auc"], label='AUC')
    plt.plot(mean_atomics.index, mean_atomics["acc"], label='ACC')
    plt.plot(mean_atomics.index, mean_atomics["f1"], label='F1')
    plt.xlabel('Num Atomics')
    plt.ylabel('Criteria')
    plt.title("Metric as mean over atomics")
    plt.suptitle(title)
    plt.legend()
    plt.show()

    # display(mean_concepts)
    plt.plot(mean_concepts.index, mean_concepts["auc"], label='AUC')
    plt.plot(mean_concepts.index, mean_concepts["acc"], label='ACC')
    plt.plot(mean_concepts.index, mean_concepts["f1"], label='F1')
    plt.xlabel('Num Concepts')
    plt.ylabel('Criteria')
    plt.title("Metric as mean over concepts")
    plt.suptitle(title)
    plt.legend()
    plt.show()


In [4]:
def initializeModel(n_concepts, static_dim, changing_dim, seq_len, output_dim, noise_std, top_k=''):
    model = original_models.CBM(static_dim = static_dim, 
                                changing_dim = changing_dim, 
                                seq_len = seq_len,
                                num_concepts = n_concepts,
                                use_indicators = True,
                                use_fixes = False,
                                use_only_last_timestep = False,
                                use_grad_norm = False,
                                noise_std = noise_std,
                                opt_lr = 1e-3,
                                opt_weight_decay = 1e-5,
                                l1_lambda=1e-3,
                                cos_sim_lambda=1e-2,
                                output_dim = output_dim,
                                top_k=top_k,
                                device = device
                                )
    model = model.to(device)
    return model

def initializeModel_with_atomics(n_atomics, n_concepts, static_dim, changing_dim, seq_len, output_dim, use_summaries_for_atomics, noise_std, top_k=''):
    model = new_models.CBM(static_dim = static_dim, 
                            changing_dim = changing_dim, 
                            seq_len = seq_len,
                            num_concepts = n_concepts,
                            num_atomics= n_atomics,
                            use_summaries_for_atomics = use_summaries_for_atomics,
                            use_indicators = True,
                            use_fixes = False,
                            use_grad_norm = False,
                            noise_std = noise_std,
                            opt_lr = 1e-3,
                            opt_weight_decay = 1e-5,
                            l1_lambda=1e-3,
                            cos_sim_lambda=1e-2,
                            output_dim = output_dim,
                            top_k=top_k,
                            device = device
                            )
    model = model.to(device)
    return model


In [5]:
auroc_metric = AUROC(task="binary").to(device)
accuracy_metric = Accuracy(task="binary").to(device)
f1_metric = F1Score(task="binary").to(device)
conf_matrix = ConfusionMatrix(task="binary").to(device)

seq_len = seq_len
changing_dim = len(changing_vars)
static_dim = len(static_names)

print(changing_dim, static_dim, seq_len)

random_seed = 1
set_seed(random_seed)


27 8 6


## Original

In [6]:
config_original = {
    "n_concepts": [4], # 20
    # "use_indicators": [True], # False
    # "use_fixes": [False], # True
    # "use_only_last_timestep": [False], # True
    # "output_dim": [2], #1
    # "use_grad_norm": [False], # "FULL", "COMPONENT_WISE"],
    "noise_std": [None, .2, .5, .7, 1, 5, 10],
}

all_config_permutations_og = list(product(*config_original.values()))
all_config_permutations_og = [dict(zip(config_original.keys(), permutation)) for permutation in all_config_permutations_og]
print(len(all_config_permutations_og))
# all_config_permutations_og

7


In [7]:
experiment_folder = "/workdir/optimal-summaries-public/_models/vasopressor/original/"
model_path = experiment_folder + "vaso_c_{n_concepts}_noise_std_{noise_std}.pt"

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [8]:
histories_original = []

for zzz in range(5):
    for i, config in enumerate(all_config_permutations_og):
        print(i, config)
        
        train_loader, val_loader, test_loader, class_weights, num_classes, changing_vars, static_names, seq_len = load_and_create_MIMIC_dataloader(output_dim = 2, batch_size = 512, random_state = 1)
        
        model = initializeModel(**config, static_dim=static_dim, changing_dim=changing_dim, seq_len=seq_len, output_dim=2)
        model.fit(train_loader, val_loader, p_weight=class_weights.to(device), save_model_path=model_path.format(**config)+f"{zzz}", max_epochs=10000)
        
        cos_sim = (model.cos_sim(model.bottleneck) / model.bottleneck.out_features).item()
        
        model.eval()
        with torch.inference_mode():
            for batch in test_loader:
                X_time, X_ind, X_static, y = extract_to(batch, device)
                probs = model.forward_probabilities(X_time, X_ind, X_static)
                
                auc = auroc_metric(probs, y).item()
                acc = accuracy_metric(probs, y).item()
                f1 = f1_metric(probs, y).item()
                # conf_matrix(probs, yb)
            auc = auroc_metric.compute().item()
            acc = accuracy_metric.compute().item()
            f1 = f1_metric.compute().item()
            # conf_matrix.plot()
            # plt.show()
            auroc_metric.reset()
            accuracy_metric.reset()
            # conf_matrix.reset()
            f1_metric.reset()
        
        history = ["original", i, model.val_losses[-1], auc, acc, f1, cos_sim]
        print(history)
        histories_original.append(np.array(history))
        
        # plot_losses(model.train_losses, model.val_losses)
        del model
        gc.collect()
        torch.cuda.empty_cache()
    
histories_original = np.array(histories_original)
histories_original.shape


0 {'n_concepts': 4, 'noise_std': None}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/original/vaso_c_4_noise_std_None.pt0
['original', 0, 0.525508463382721, 0.9068102240562439, 0.8282992839813232, 0.8375415802001953, 0.0034812933299690485]
1 {'n_concepts': 4, 'noise_std': 0.2}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/original/vaso_c_4_noise_std_0.2.pt0
['original', 1, 0.5295618772506714, 0.920878529548645, 0.84503173828125, 0.8515534996986389, 0.002568579278886318]
2 {'n_concepts': 4, 'noise_std': 0.5}


...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/original/vaso_c_4_noise_std_5.pt4
['original', 5, 0.6161332130432129, 0.8891679644584656, 0.8625364899635315, 0.8625364899635315, 0.006982322316616774]
6 {'n_concepts': 4, 'noise_std': 10}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/original/vaso_c_4_noise_std_10.pt4
['original', 6, 0.6169614791870117, 0.8925348520278931, 0.8625364899635315, 0.8625364899635315, 0.0028808594215661287]


(35, 7)

In [9]:
# plot
# plot_metrics(histories_original, n_concepts_list)


## Atomics

In [10]:
config_atomics = {
    "n_atomics": [10], # 30
    "n_concepts": [4], # 20
    # "use_indicators": [True], # False
    # "use_fixes": [False], # True
    # "output_dim": [2], #1
    "use_summaries_for_atomics": [True], # False
    # "use_grad_norm": [False], # "FULL", "COMPONENT_WISE"],
    "noise_std": [None, .2, .5, .7, 1, 5, 10],
}

all_config_permutations_atomics = list(product(*config_atomics.values()))
all_config_permutations_atomics = [dict(zip(config_atomics.keys(), permutation)) for permutation in all_config_permutations_atomics]
print(len(all_config_permutations_atomics))
# all_config_permutations_atomics

7


In [11]:
experiment_folder = "/workdir/optimal-summaries-public/_models/vasopressor/atomics/"
model_path = experiment_folder + "vaso_a_{n_atomics}_c_{n_concepts}_summaries4atomics_{use_summaries_for_atomics}_noise_std_{noise_std}.pt"

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [12]:
history_atomics = []

for zzz in range(5):
    for i, config in enumerate(all_config_permutations_atomics):
        print(i, config)
        
        train_loader, val_loader, test_loader, class_weights, num_classes, changing_vars, static_names, seq_len = load_and_create_MIMIC_dataloader(output_dim = 2, batch_size = 512, random_state = 1)
        
        model = initializeModel_with_atomics(**config, static_dim=static_dim, changing_dim=changing_dim, seq_len=seq_len, output_dim=2)
        model.fit(train_loader, val_loader, p_weight=class_weights.to(device), save_model_path=model_path.format(**config)+f"{zzz}", max_epochs=10000)
        
        cos_sim = ((model.cos_sim(model.layer_to_concepts) + model.cos_sim(model.layer_time_to_atomics)) 
                / (model.layer_to_concepts.out_features + model.layer_time_to_atomics.out_features)).item()
        
        model.eval()
        with torch.inference_mode():
            for batch in test_loader:
                X_time, X_ind, X_static, y = extract_to(batch, device)
                probs = model.forward_probabilities(X_time, X_ind, X_static)
                
                auc = auroc_metric(probs, y).item()
                acc = accuracy_metric(probs, y).item()
                f1 = f1_metric(probs, y).item()
                # conf_matrix(probs, yb)
            auc = auroc_metric.compute().item()
            acc = accuracy_metric.compute().item()
            f1 = f1_metric.compute().item()
            # conf_matrix.plot()
            # plt.show()
            auroc_metric.reset()
            accuracy_metric.reset()
            # conf_matrix.reset()
            f1_metric.reset()

        history = ["atomics", i, model.val_losses[-1], auc, acc, f1, cos_sim]
        print(history)
        history_atomics.append(np.array(history))
        
        # plot_losses(model.train_losses, model.val_losses)
        del model
        gc.collect()
        torch.cuda.empty_cache()

    
history_atomics = np.array(history_atomics)
history_atomics.shape


0 {'n_atomics': 10, 'n_concepts': 4, 'use_summaries_for_atomics': True, 'noise_std': None}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/atomics/vaso_a_10_c_4_summaries4atomics_True_noise_std_None.pt0
['atomics', 0, 0.5391227602958679, 0.9119085073471069, 0.8328471183776855, 0.8414197564125061, 0.006680180784314871]
1 {'n_atomics': 10, 'n_concepts': 4, 'use_summaries_for_atomics': True, 'noise_std': 0.2}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/atomics/vaso_a_10_c_4_summaries4atomics_True_noise_std_0.2.pt0
['atomics', 1, 0.5346981287002563, 0.9183363914489746, 0.84425950050354, 0.8513635396957397, 0.009473815560340881]
2 {'n_atomics': 10, 'n_concepts': 4, 'use_summaries_for_atomics': True, 'noise_std': 0.5}


...

Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/atomics/vaso_a_10_c_4_summaries4atomics_True_noise_std_None.pt2
['atomics', 0, 0.5278989672660828, 0.9155356884002686, 0.8387678265571594, 0.8469744920730591, 0.008884754031896591]
1 {'n_atomics': 10, 'n_concepts': 4, 'use_summaries_for_atomics': True, 'noise_std': 0.2}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/atomics/vaso_a_10_c_4_summaries4atomics_True_noise_std_0.2.pt2
['atomics', 1, 0.5326784253120422, 0.9183775782585144, 0.8496653437614441, 0.8562756180763245, 0.007676935754716396]
2 {'n_atomics': 10, 'n_concepts': 4, 'use_summaries_for_atomics': True, 'noise_std': 0.5}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/vasopressor/atomics/vaso_a_10_c_4_summaries4atomics_True_noise_std_0.5.pt2


In [ ]:
columns=["type", "config", "val_loss", "auc", "acc", "f1", "cos_sim"]
dtypes = {'type': str, 'config': int, 'val_loss': float, 'auc': float, 'acc': float, 'f1': float, 'cos_sim': float}

df_og = pd.DataFrame(histories_original, columns=columns).astype(dtypes)
df_og = pd.concat([df_og, pd.DataFrame(all_config_permutations_og)], axis=1)

df_atomics = pd.DataFrame(history_atomics, columns=columns).astype(dtypes)
df_atomics = pd.concat([df_atomics, pd.DataFrame(all_config_permutations_atomics)], axis=1)

result_df = pd.concat([df_og, df_atomics], ignore_index=True)
# result_df

In [ ]:
columns = ["type", "config", "val_loss", "auc", "acc", "f1", "cos_sim"]
dtypes = {'type': str, 'config': int, 'val_loss': float, 'auc': float, 'acc': float, 'f1': float, 'cos_sim': float}

df_og = pd.DataFrame(histories_original, columns=columns).astype(dtypes)
df_og = df_og.join(pd.DataFrame(all_config_permutations_og), on="config", how="left")

df_atomics = pd.DataFrame(history_atomics, columns=columns).astype(dtypes)
df_atomics = df_atomics.join(pd.DataFrame(all_config_permutations_atomics), on="config", how="left")

result_df = pd.concat([df_og, df_atomics], ignore_index=True)
# result_df


In [ ]:
for col in result_df.columns[3:6]:
    baseline = result_df[(result_df['type'] == 'original') & (result_df['config'] == 0)][col].values[0]
    print(col, baseline)
    result_df[f'{col}_abs_imp'] = result_df[col] - baseline
    # result_df[f'{col}_rel_imp'] = result_df[f'{col}_abs_imp'] / baseline
# result_df

auc 0.9145365357398987
acc 0.8379097580909729
f1 0.8460096120834351


In [ ]:
pd.set_option('display.max_rows', 500)
result_df.sort_values(by='acc', ascending=False)
# atomics: atomics, concepts, use_indicators, use_fixes, output_dim, use_summaries_for_atomics
# original: concepts, use_indicators, use_fixes, output_dim, use_only_last_timestep


,type,config,val_loss,auc,acc,f1,cos_sim,n_concepts,noise_std,n_atomics,use_summaries_for_atomics,auc_abs_imp,acc_abs_imp,f1_abs_imp
7,atomics,0,0.616499,0.869403,0.862536,0.862536,0.006716,4,0.1,10.0,True,-0.045133,0.024627,0.016527
4,original,0,0.523275,0.918397,0.843573,0.850586,0.005025,4,0.1,NaN,NaN,0.003860,0.005663,0.004576
9,atomics,0,0.528923,0.915159,0.842114,0.849698,0.007062,4,0.1,10.0,True,0.000623,0.004205,0.003688
1,original,0,0.522486,0.917275,0.840312,0.847472,0.001381,4,0.1,NaN,NaN,0.002738,0.002403,0.001462
8,atomics,0,0.532362,0.913648,0.838682,0.846506,0.008808,4,0.1,10.0,True,-0.000889,0.000772,0.000496
5,atomics,0,0.533410,0.914100,0.838167,0.845965,0.008016,4,0.1,10.0,True,-0.000437,0.000257,-0.000044
0,original,0,0.523795,0.914537,0.837910,0.846010,0.002108,4,0.1,NaN,NaN,0.000000,0.000000,0.000000
2,original,0,0.522644,0.915804,0.836451,0.844560,0.002697,4,0.1,NaN,NaN,0.001267,-0.001459,-0.001449
6,atomics,0,0.531795,0.912748,0.836108,0.844209,0.006832,4,0.1,10.0,True,-0.001789,-0.001802,-0.001801
3,original,0,0.520821,0.906899,0.831732,0.839643,0.002898,4,0.1,NaN,NaN,-0.007638,-0.006178,-0.006366


In [ ]:
for key in sorted(set(list(all_config_permutations_og[0].keys()) + list(all_config_permutations_atomics[0].keys()))):
    display(result_df.groupby(["type", key], dropna=False)[["auc", "acc", "f1"]].mean())

display(result_df.groupby("type")[["auc", "acc", "f1"]].mean())

,,auc,acc,f1
type,n_atomics,,,
atomics,10.0,0.905012,0.843522,0.849783
original,NaN,0.914582,0.837996,0.845654


,,auc,acc,f1
type,n_concepts,,,
atomics,4,0.905012,0.843522,0.849783
original,4,0.914582,0.837996,0.845654


,,auc,acc,f1
type,noise_std,,,
atomics,0.1,0.905012,0.843522,0.849783
original,0.1,0.914582,0.837996,0.845654


,,auc,acc,f1
type,use_summaries_for_atomics,,,
atomics,True,0.905012,0.843522,0.849783
original,NaN,0.914582,0.837996,0.845654


,auc,acc,f1
type,,,
atomics,0.905012,0.843522,0.849783
original,0.914582,0.837996,0.845654


## Optimization

In [ ]:
# feature weights
n_concepts = 4

model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, num_classes)
model.fit(train_loader, val_loader, class_weights, model_path.format(n_concepts), 1000)

for batch_idx, (Xb, yb) in enumerate(test_loader):
    Xb, yb = Xb.to(device), yb.to(device)
    probs = model.forward_probabilities(Xb)
    
    auc = auroc_metric(probs, yb).item()
    acc = accuracy_metric(probs, yb).item()
    conf_matrix(probs, yb)
auc = auroc_metric.compute().item()
acc = accuracy_metric.compute().item()
conf_matrix.plot()
auroc_metric.reset()
accuracy_metric.reset()
conf_matrix.reset()

print("AUC", auc)
print("ACC", acc)

In [ ]:
for name, param in model.named_parameters():
    if "bottleneck.weight" in name:
        bottleneck_weights = param
feature_weights = bottleneck_weights.cpu().detach().numpy()

feature_weights.shape

In [ ]:
# visualize weight magnitudes
for c in range(n_concepts):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    inds = np.argsort(-np.abs(feature_weights[c]))[:100]
    ax.bar(np.arange(1,101),np.abs(feature_weights[c])[inds])
    ax.set_xlabel("Top 100 features")
    ax.set_ylabel("abs value of feature coefficient")
    plt.show()


In [ ]:
# get 90th percentile of feature weights
sum90p = np.sum(np.abs(feature_weights), axis=-1)*0.90
sum90p.shape


In [ ]:
# get top K indizes
top_k_inds = []
for c in range(n_concepts):
    topkinds_conc = []
    curr_sum = 0
    inds = np.argsort(-np.abs(feature_weights[c])) #desc
    sorted_weights = feature_weights[c][inds]
    
    for ind, weight in zip(inds, sorted_weights):
        curr_sum += abs(weight)
        if curr_sum <= sum90p[c]:
            topkinds_conc.append(ind)
        else:
            break
    
    # if selects less than 10, choose 10 best
    if len(topkinds_conc) < 10:
        topkinds_conc = np.argsort(-np.abs(feature_weights[c]))[:10].tolist()
    
    top_k_inds.append(topkinds_conc)

top_k_inds

In [ ]:
# write top k inds to csv
filename = experiment_folder + "top-k/top_k_inds_c{}.csv".format(n_concepts)

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    # writing the data rows 
    csvwriter.writerows(top_k_inds)


In [ ]:
V = 13 + 1
T = seq_len + 1
print(T)
vars_ = [i for i in range(1,V)] + [str(i) + "_ind" for i in range(1,V)]
print(len(vars_))
data_cols = [["feat_{}_time_{}".format(v, t) for v in vars_] for t in range(1, T)]
flattened_data_cols = [col for sublist in data_cols for col in sublist]
print(len(flattened_data_cols))
flattened_data_cols

In [ ]:

for c, _list in enumerate(top_k_inds):
    for ind in _list:
        name, summary = getConcept(flattened_data_cols, input_dim, changing_dim, int(ind))
        print(f"Concept {c}: ID {ind}, Feature {name}, Summary {summary}")


In [ ]:
greedy_results = greedy_selection(auroc_metric, test_loader, top_k_inds, model, track_metrics={"acc": accuracy_metric})
greedy_results.head(10)

In [ ]:
top_k_csv_file = experiment_folder + "top-k/bottleneck_r{}_c{}_topkinds.csv".format(random_seed, n_concepts)

# writing to csv file
with open(top_k_csv_file, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(greedy_results.columns)
    # writing the data rows 
    for row in greedy_results.itertuples(index=False):
        csvwriter.writerow(list(row))


In [ ]:
sorted_ = greedy_results.sort_values(["Concept", "ID"])

for row in sorted_.itertuples(index=False):
    name, summary = getConcept(flattened_data_cols, input_dim, changing_dim, row[1])
    print(f"Concept {row[2]}: ID {row[1]}, Feature {name}, Summary {summary}")

In [ ]:
plt.plot(greedy_results["Score"], label = f"AUC {greedy_results['Score'].values[-1]:.3f}")
plt.plot(greedy_results["acc"], label = f"ACC {greedy_results['acc'].values[-1]:.3f}")

plt.xlabel('Num Concepts')
plt.ylabel('Criteria')
plt.title('Plot of Concepts vs Criteria')

plt.legend()
plt.show()


In [ ]:
top_k_csv_file = "/workdir/optimal-summaries-public/_models/arabic/multiclass/top-k/bottleneck_r1_c6_topkinds.csv"
n_concepts = 6
model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, num_classes, top_k=top_k_csv_file)
# model.fit(train_loader, val_loader, weights, model_path.format(n_concepts), 1000)

model.eval()
with torch.no_grad():
    for batch_idx, (Xb, yb) in enumerate(test_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        probs = model.forward_probabilities(Xb)
        
        auc = auroc_metric(probs, yb).item()
        acc = accuracy_metric(probs, yb).item()
    auc = auroc_metric.compute().item()
    acc = accuracy_metric.compute().item()
    auroc_metric.reset()
    accuracy_metric.reset()

print(auc)
print(acc)


In [ ]:
model.fit(train_loader, val_loader, class_weights, save_model_path="/workdir/optimal-summaries-public/_models/arabic/multiclass/top-k/arabic_c6_finetuned.pt", max_epochs=3000, patience=100)

model.eval()
with torch.no_grad():
    for batch_idx, (Xb, yb) in enumerate(test_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        probs = model.forward_probabilities(Xb)
        
        auc = auroc_metric(probs, yb)
        acc = accuracy_metric(probs, yb)
    auc = auroc_metric.compute().item()
    acc = accuracy_metric.compute().item()
    auroc_metric.reset()
    accuracy_metric.reset()
    
print(auc)
print(acc)


In [ ]:

plt.plot(model.val_losses)
plt.show()